colabで回すことを前提にしています

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
! pip install googletrans==4.0.0rc1

In [ ]:
# fileの移し替え
!cp -r /content/drive/MyDrive/dotfiles_for_colab/.vscode/ /content/
!mkdir /root/.kaggle/
!cp /content/drive/MyDrive/dotfiles_for_colab/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [ ]:
# kaggleからデータのダウンロード
# !mkdir /content/input/ 
!kaggle competitions download -c nbme-score-clinical-patient-notes -p /content/input/nbme-score-clinical-patient-notes/
!unzip /content/input/nbme-score-clinical-patient-notes/nbme-score-clinical-patient-notes.zip -d /content/input/nbme-score-clinical-patient-notes/
!rm /content/input/nbme-score-clinical-patient-notes/nbme-score-clinical-patient-notes.zip

  0% 0.00/9.83M [00:00<?, ?B/s]
100% 9.83M/9.83M [00:00<00:00, 122MB/s]
Archive:  /content/input/nbme-score-clinical-patient-notes/nbme-score-clinical-patient-notes.zip
replace /content/input/nbme-score-clinical-patient-notes/features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# kaggleからデータのダウンロード
!kaggle datasets download -d nbroad/deberta-v2-3-fast-tokenizer -p /content/input/deberta-v2-3-fast-tokenizer/
!unzip /content/input/deberta-v2-3-fast-tokenizer/deberta-v2-3-fast-tokenizer.zip -d /content/input/deberta-v2-3-fast-tokenizer/
!rm /content/input/deberta-v2-3-fast-tokenizer/deberta-v2-3-fast-tokenizer.zip

In [ ]:
import pandas as pd
import ast

def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features

df_train = pd.read_csv('./input/nbme-score-clinical-patient-notes/train.csv')
df_train['annotation'] = df_train['annotation'].map(lambda x: ast.literal_eval(x))
df_train['location'] = df_train['location'].map(lambda x: ast.literal_eval(x))

features = pd.read_csv('./input/nbme-score-clinical-patient-notes/features.csv')
features = preprocess_features(features)

patient_notes = pd.read_csv('./input/nbme-score-clinical-patient-notes/patient_notes.csv')

print(f"df_train.shape: {df_train.shape}")
display(df_train.head())
print(f"features.shape: {features.shape}")
display(features.head())
print(f"patient_notes.shape: {patient_notes.shape}")
display(patient_notes.head())
df_train = df_train.merge(features, on=['feature_num', 'case_num'], how='left')
df_train = df_train.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(df_train.head())

In [ ]:
from googletrans import Translator

def retranslator(text, lang):
    '''英語から他言語に翻訳したあと再度英語に翻訳し直してデータの水増しを狙う
    '''
    translator = Translator()
    translated = translator.translate(text, src='en', dest=lang).text
    retranslated = translator.translate(translated, src=lang, dest='en').text
    return translated, retranslated

In [ ]:
unique_feature_text=df_train['feature_text'].unique()

In [ ]:
from collections import defaultdict
from tqdm.auto import tqdm
lang_ls=['ja', 'ko', 'ru', 'ca']
feature_text_retranslated=defaultdict(lambda: list())
for ft in tqdm(unique_feature_text):
    for lang in lang_ls:
      translated, retranslated=retranslator(ft.replace('-', ' '), lang)
      feature_text_retranslated[lang].append(retranslated)

  0%|          | 0/131 [00:00<?, ?it/s]

In [ ]:
df_retranslated=pd.DataFrame(feature_text_retranslated, columns=lang_ls).assign(feature_text=unique_feature_text)
df_retranslated

,ja,ko,ru,ca,feature_text
0,History of families of MI or family of myocard...,MI in myocardial infarction MI or the history ...,Family History of MI or Family History Myocard...,Family History of Mi or Family History of Myoc...,Family-history-of-MI-OR-Family-history-of-myoc...
1,History of family of thyroid disorder,Family force of thyroid disorder,Family history of thyroid disorder,Family history of thyroid disorder,Family-history-of-thyroid-disorder
2,Chest pressure,Chest pressure,Chest pressure,Chest pressure,Chest-pressure
3,Intermittent symptoms,Intermittent symptoms,Intermittent symptoms,Intermittent symptoms,Intermittent-symptoms
4,Light head,Light,Lighthead.,Light headed,Lightheaded
...,...,...,...,...,...
126,Nemorrhabitic vaccine vaccine,Can not know the sterile sterilization vaccine,Meningococcal Vaccine Status Unknown,State of the unknown meningococcal vaccine,Meningococcal-vaccine-status-unknown
127,Family history of migraine,Family history of migraine,Family history of migraine,Family History of Migraines,Family-history-of-migraines
128,Fingering,life span,Photophobia,Photophobia,Photophobia
129,There is no known disease contact,There is no known disease contact,No known diseases contacts,There are no contacts of known diseases,No-known-illness-contacts


In [ ]:
df_retranslated.to_csv('drive/MyDrive/df_retranslated.csv', index=False)
df_retranslated.to_pickle('drive/MyDrive/df_retranslated.pkl')

In [ ]:
raise ValueError()

ValueError: ignored

In [ ]:
retranslator('fathre-MI', 'ja')

In [ ]:
retranslator('Family history of MI OR Family history of myocardial infarction', 'ja')

In [ ]:
retranslator('Family history of thyroid disorder', 'ja')

In [ ]:
retranslator('Family history of thyroid disorder', 'ru')

In [ ]:
retranslator('Family history of thyroid disorder', 'ca')

In [ ]:
retranslator('Family history of thyroid disorder', 'zh-CN')

In [ ]:
retranslator('Family history of thyroid disorder', 'ko')